In [28]:
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple
import os

In [29]:
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder {folder_path} created successfully.")
    else:
        print(f"Folder {folder_path} already exists.")


create_folder_if_not_exists("plots")
create_folder_if_not_exists("errors")

Folder plots already exists.
Folder errors already exists.


In [30]:
import csv


def save_to_csv(filename, data):
    filename = "errors/" + filename + ".csv"
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data)

In [31]:
def multiply_matrices(A, B):
    result = np.zeros((len(A), len(B[0])), dtype=np.float64)
    for i in range(len(A)):
        for j in range(len(B[0])):
            for k in range(len(B)):
                result[i][j] += A[i][k] * B[k][j]
    return result


def residual_stop_criterium(A, x, b, rho):
    def norm(vector):
        max_value = -np.inf
        for value in vector:
            if np.abs(value) > max_value:
                max_value = np.abs(value)
        return max_value

    return norm(multiply_matrices(A, x) - b) < rho


def difference_stop_criterium(x, x_new, rho):
    def norm(vector):
        max_value = -np.inf
        for value in vector:
            if np.abs(value) > max_value:
                max_value = np.abs(value)
        return max_value

    return norm(x - x_new) < rho

In [32]:
def get_random_sign():
    return np.random.choice([-1, 1])


def get_x(n, variable_type):
    x_ = np.zeros(n, dtype=variable_type)
    for i in range(n):
        x_[i] = variable_type(variable_type(get_random_sign()))
    return x_


def get_b(A, x, n, variable_type):
    b = np.zeros(n, dtype=variable_type)
    for i in range(n):
        for j in range(n):
            b[i] += A[i][j] * x[j]
        print()
    return b

In [33]:
k = np.float64(6)
m = np.float64(4)
one = np.float64(1)


def get_A(size):
    global k, m, one
    A = np.zeros((size, size), dtype=np.float64)
    for i in range(size):
        for j in range(size):
            i_ = np.float64(i)
            j_ = np.float64(j)

            if i == j:
                A[i][j] = k
            elif j > i:
                A[i][j] = (-one) ** (j_ + one) * m / (j_ + one)
            elif j + 1 == i:
                A[i][j] = m / (i_ + one)
            else:
                A[i][j] = np.float64(0)
    return A


def get_D(n):
    global k, m, one
    D = np.zeros((n, n), dtype=np.float64)
    for i in range(n):
        D[i][i] = k
    return D


def get_D_inverted(n):
    global k, m, one
    D = np.zeros((n, n), dtype=np.float64)
    k_ = np.float64(k)
    for i in range(n):
        D[i][i] = one / k_
    return D


def get_negative_D_inverted(n):
    global k, m, one
    D = np.zeros((n, n), dtype=np.float64)
    k_ = np.float64(k)
    minus_one = np.float64(-1)
    for i in range(n):
        D[i][i] = minus_one / k_
    return D


def get_L(n):
    global k, m, one
    L = np.zeros((n, n), dtype=np.float64)
    for i in range(n):
        for j in range(n):
            i_ = np.float64(i)
            j_ = np.float64(j)
            if i == j + 1:
                L[i][j] = m / (i_ + one)
    return L


def get_U(n):
    global k, m, one
    U = np.zeros((n, n), dtype=np.float64)
    for i in range(n):
        for j in range(n):
            i_ = np.float64(i)
            j_ = np.float64(j)
            if j > i:
                U[i][j] = (-one) ** (j_ + one) * m / (j_ + one)
    return U


def get_M(n):
    U = get_U(n)
    L = get_L(n)
    negative_D_inverted = get_negative_D_inverted(n)
    M = np.dot(negative_D_inverted, L + U)
    return M


In [43]:
gowno = np.float64(1)
print( isinstance(gowno, np.float64) )

True


In [77]:
def multiply_matrices(A, B):
    if not isinstance(B[0], np.ndarray):
        result = np.zeros(len(A), dtype=np.float64)
        for i in range(len(A)):
            for j in range(len(B)):
                result[i] += A[i][j] * B[j]
        return result
    else:
        result = np.zeros((len(A), len(B[0])), dtype=np.float64)
        for i in range(len(A)):
            for j in range(len(B[0])):
                for k in range(len(B)):
                    result[i][j] += A[i][k] * B[k][j]
        return result


def norm(vector):
    max_value = -np.inf
    for value in vector:
        if np.abs(value) > max_value:
            max_value = np.abs(value)
    return max_value


class Jacobi:
    def __init__(self, b, ceil_limitation, rho, stop_criterium, x_start):
        self.n = len(b)
        self.A = get_A(self.n)
        self.errors = []
        self.b = b
        self.ceil_limitation = ceil_limitation
        self.M = get_M(self.n)
        self.b = b
        self.N = get_D_inverted(self.n)
        self.rho = rho
        self.stop_criterium = stop_criterium
        self.x = x_start
        if stop_criterium != "res" and stop_criterium != "diff":
            raise Exception("Wrong stop criterium")

    def residual_stop_criterium(self):
        return norm(multiply_matrices(self.A, self.x) - self.b) < self.rho

    def difference_stop_criterium(self, x_next):
        return norm(x_next - self.x) < self.rho

    def calculate(self):
        for i in range(self.ceil_limitation):
            x_next = multiply_matrices(self.M, self.x) + multiply_matrices(self.N, self.b)
            self.errors.append(norm(x_next - self.x))
            if self.stop_criterium == "res":
                if self.residual_stop_criterium():
                    return x_next, self.errors, i + 1
            elif self.stop_criterium == "diff":
                if self.difference_stop_criterium(x_next):
                    return x_next, self.errors, i + 1
            self.x = x_next
        return x_next, self.errors, self.ceil_limitation


In [78]:
import numpy as np


def generate_random_system(num_equations, num_variables):
    A = np.random.rand(num_equations, num_variables)
    b = np.random.rand(num_equations)
    return A, b


def test_jacobi_vs_np_solve(n):
    for i in range(1000):
        _, b = generate_random_system(n, n)
        A = get_A(n)
        np_solution = np.linalg.solve(A, b)
        jacobi_solver = Jacobi(b=b, ceil_limitation=1000, rho=0.0000000001, stop_criterium="diff",
                               x_start=np.zeros(n))
        jacobi_solution = jacobi_solver.calculate()[0]

        if np.allclose(np_solution, jacobi_solution):
            print("The solutions match!")
        else:
            print(f"The solutions do not match. np_solution = {np_solution}, jacobi_solution = {jacobi_solution}")
            return


# Test the Jacobi class against np.linalg.solve
# test_jacobi_vs_np_solve(10)


[0.13169643 0.38169643 0.41517857]


In [ ]:
def spectral_radius(matrix):
    eigenvalues = np.abs(np.linalg.eigvals(matrix))
    return np.max(eigenvalues)


'''for n in range(3, 150):
    M = get_M(n)
    print(f"n = {n}, spectral radius = {spectral_radius(M)}")'''  #%%


def spectral_radius(matrix):
    eigenvalues = np.abs(np.linalg.eigvals(matrix))
    return np.max(eigenvalues)


'''for n in range(3, 150):
    M = get_M(n)
    print(f"n = {n}, spectral radius = {spectral_radius(M)}")'''